In [26]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [32]:
query = """
SELECT
    s.name AS zugewiesene_station,
    p.vorname || ' ' || p.nachname AS patient_name,
    p.geburtsdatum,
    a.vorname || ' ' || a.nachname AS arzt_name,
    b.beschreibung AS behandlung_beschreibung,
    m.wirkstoff
FROM arzt a
JOIN arzt_event ae ON a.arzt_id = ae.arzt_id
JOIN event e ON ae.event_id = e.event_id
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
JOIN event_medikament em ON e.event_id = em.event_id
JOIN medikament m ON em.medikament_id = m.medikament_id
JOIN station s ON s.station_id = a.station_id
WHERE a.arzt_id = 1
  AND b.abgeschlossen = false;
"""
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)


('Chirurgie', 'Mauro Tobler', datetime.date(1930, 4, 26), 'Anke Bolliger', 'Standardbehandlung', 'Acetylsalicylsäure')
('Chirurgie', 'Mauro Tobler', datetime.date(1930, 4, 26), 'Anke Bolliger', 'Standardbehandlung', 'Amoxicillin')
('Chirurgie', 'Mimoza Meyer', datetime.date(1939, 4, 30), 'Anke Bolliger', 'Standardbehandlung', 'Acetylsalicylsäure')
('Chirurgie', 'Mimoza Meyer', datetime.date(1939, 4, 30), 'Anke Bolliger', 'Standardbehandlung', 'Amoxicillin')
('Chirurgie', 'Leticia Stadelmann', datetime.date(1947, 3, 12), 'Anke Bolliger', 'Standardbehandlung', 'Acetylsalicylsäure')
('Chirurgie', 'Leticia Stadelmann', datetime.date(1947, 3, 12), 'Anke Bolliger', 'Standardbehandlung', 'Amoxicillin')
('Chirurgie', 'Maik Hasler', datetime.date(1961, 10, 31), 'Anke Bolliger', 'Standardbehandlung', 'Acetylsalicylsäure')
('Chirurgie', 'Maik Hasler', datetime.date(1961, 10, 31), 'Anke Bolliger', 'Standardbehandlung', 'Amoxicillin')
('Chirurgie', 'Mauro Tobler', datetime.date(1930, 4, 26), 'Anke 

In [29]:
query3 = """
-- shows which station uses which drug how often
SELECT
    s.name as Station,
    m.wirkstoff as Medikament,
    COUNT(*) AS verwendungen
FROM behandlung_medikament bm
JOIN medikament m ON bm.medikament_id = m.medikament_id
JOIN behandlung b ON bm.behandlungs_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
JOIN station s ON p.station_id = s.station_id
GROUP BY s.name, m.wirkstoff
ORDER BY s.name, verwendungen DESC;
"""

In [30]:
query4 = """
-- zeigt wie viele behandlungen ein arzt hat

SELECT 
    a.Arzt_ID,
    a.Name AS Arztname,
    COUNT(ab.Behandlungs_ID) AS Anzahl_Behandlungen
FROM Arzt a
LEFT JOIN Arzt_Behandlung ab 
    ON a.Arzt_ID = ab.Arzt_ID
GROUP BY a.Arzt_ID, a.Name
ORDER BY COUNT(ab.Behandlungs_ID) DESC;
"""

In [31]:
query5 = """
-- anzahl belegte betten
SELECT 
    s.Station_ID,
    s.Name AS Station,
    COUNT(*) AS Gesamtbetten,
    COUNT(CASE WHEN b.Status = 'belegt' THEN 1 END) AS Belegte_Betten,
    COUNT(CASE WHEN b.Status = 'frei' THEN 1 END) AS Freie_Betten
FROM Station s
JOIN Bett b ON b.Station_ID = s.Station_ID
GROUP BY s.Station_ID, s.Name
ORDER BY s.Station_ID;
"""